In [ ]:
# Execute before using this notebook if using google colab
kernel = str(get_ipython())

if 'google.colab' in kernel:    
    !wget https://raw.githubusercontent.com/fredzett/rmqa/master/utils.py -P local_modules -nc 
    !npx degit fredzett/rmqa/data data
    import sys
    sys.path.append('local_modules')

In [ ]:
import numpy as np 
import scipy.stats as stats

from utils import Datasets, plot_bar, plot_line

# Exercise 1

A (regular) dice is thrown 10 times. Answer the following questions:

- what is the probability that a six is shown exactly 5 times?

- what is the probability that a six is shown less than 3 times?

- what is the expected number of sixes shown?

- what is the variance of number of sixes (if you were to repeat the experiment multiple times)?

Note: a regular dice has 6 eyes (1, 2, 3, 4, 5, 6) with equal probability of being shown

## Solution 1

# Exercise 2

A class has to take a multiple choice test with $20$ questions. Each question has $5$ possible answers of which only $1$ is correct. You need to answer at least $10$ questions correctly to pass the exam. 

- how many correct answers would you expect to receive by purely guessing?

- if $1.000$ students take the test, all of which will just guess the answers. How many of them would you expect to pass?

## Solution 2

# Exercise 3

A random variable $X$ is binomially distributed with a mean of $6$ and a variance of $4.2$. 

- write a function that calculates $n$ and $p$ from mean and variance

- what is $P(X \leq 6)$?

## Solution 3

# Exercise 4

You play a game with your friend. The game works as follows:

- You filp a coin. Head = you win, Tail = your friend wins  
    *(assume that it is a fair coin with $50\%$ probability of head or tail)*

- The person that wins 4 flips wins the overall game

Questions:

- what is the probability that the game lasts 4 games

- what is the probability that the game lasts 5 games

- what is the probability that the game lasts 7 games



## Solution 4

# Exercise 5

The Value at Risk (VaR) is a measure that is used in (financial) risk modelling to quantify the level of financial risk. It is very commonly used in commercial and investment banks to quantify the maximum loss potential over a given period. 

Example: a **daily 95%-VaR of 50 Million TYR** means that only in 5% of the days the losses will exceed 50 Million TYR.

**Question:** what is the daily 99%-VaR or holding a position of 100 Million TYR in the German Dax? I.e. determine the loss threshold that only occurs in 1% of the days.

(Note: assume that daily returns are normally distributed)

In [ ]:
prices = Datasets.dax_daily()

## Solution 5